In [3]:
import sys
!{sys.executable} -m pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 2.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 694.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 179.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 3.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.4 MB/s eta 0:00:0

In [2]:
import torch
import numpy as np
import cv2
from time import time

ModuleNotFoundError: No module named 'torch'

In [4]:
class ObjectDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self, url, out_file="Labeled_Video.avi"):
        """
        Initializes the class with youtube url and output file.
        :param url: Has to be as youtube URL,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = url
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        # play = pafy.new(self._URL).streams[-1]
        # assert play is not None
        return cv2.VideoCapture(self.url)

    def load_model(self):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        return labels, cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        player = self.get_video_from_url()
        assert player.isOpened()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        while True:
            start_time = time()
            ret, frame = player.read()
            assert ret
            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(frame)

In [9]:
# Create a new object and execute.
a = ObjectDetection('../input_1080p.mp4')
a()

Using cache found in C:\Users\ashys/.cache\torch\hub\ultralytics_yolov5_master
requirements: YOLOv5 requirement "tqdm>=4.64.0" not found, attempting AutoUpdate...

requirements: 1 package updated per C:\Users\ashys\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-2-12 Python-3.9.7 torch-1.8.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Frames Per Second : 1.7825311942959
Frames Per Second : 4.566210045662101
Frames Per Second : 4.405286343612334
Frames Per Second : 3.861003861003861
Frames Per Second : 4.184100418410042
Frames Per Second : 4.694835680751174
Frames Per Second : 4.149377593360996
Frames Per Second : 3.8910505836575875
Frames Per Second : 3.663003663003663
Frames Per Second : 4.484304932735426
Frames Per Second : 4.694835680751174
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.237288135593221
Frames Per Second : 4.237288135593221
Frames Per Second : 4.405286343612334
Frames Per Second : 4.405286343612334
Frames Per Second : 4.25531914893617
Frames Per Second : 4.504504504504505
Frames Per Second : 4.405286343612334
Frames Per Second : 4.545454545454546
Frames Per Second : 4.566210045662101
Frames Per Second : 4.065040650406504
Frames Per Second : 4.566210045662101
Frames Per Second : 4.484304932735426
Frames Per Second : 4.464285714285714
Frames Per Second : 4.484304932735426
Frames Per Se

Frames Per Second : 4.3478260869565215
Frames Per Second : 4.329004329004329
Frames Per Second : 4.504504504504505
Frames Per Second : 4.237288135593221
Frames Per Second : 4.201680672268908
Frames Per Second : 3.773584905660377
Frames Per Second : 4.032258064516129
Frames Per Second : 4.405286343612334
Frames Per Second : 4.405286343612334
Frames Per Second : 4.184100418410042
Frames Per Second : 4.566210045662101
Frames Per Second : 4.405286343612334
Frames Per Second : 4.273504273504273
Frames Per Second : 4.484304932735426
Frames Per Second : 4.524886877828054
Frames Per Second : 4.310344827586206
Frames Per Second : 3.7593984962406015
Frames Per Second : 3.6101083032490973
Frames Per Second : 4.329004329004329
Frames Per Second : 4.11522633744856
Frames Per Second : 4.405286343612334
Frames Per Second : 4.132231404958678
Frames Per Second : 4.504504504504505
Frames Per Second : 4.273504273504273
Frames Per Second : 4.149377593360996
Frames Per Second : 3.4602076124567476
Frames Pe

Frames Per Second : 4.291845493562231
Frames Per Second : 4.739336492890995
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.237288135593221
Frames Per Second : 4.184100418410042
Frames Per Second : 4.273504273504273
Frames Per Second : 4.065040650406504
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.016064257028113
Frames Per Second : 4.310344827586206
Frames Per Second : 3.623188405797101
Frames Per Second : 4.25531914893617
Frames Per Second : 4.201680672268908
Frames Per Second : 4.032258064516129
Frames Per Second : 3.623188405797101
Frames Per Second : 3.9215686274509802
Frames Per Second : 3.5087719298245617
Frames Per Second : 3.8167938931297707
Frames Per Second : 3.717472118959108
Frames Per Second : 3.9682539682539684
Frames Per Second : 3.597122302158273
Frames Per Second : 4.081632653061225
Frames Per Second : 4.524886877828054
Frames Per Second : 4.444444444444445
Frames Per Second : 3.8910505836575875
Frames Per Second : 4.081632653061225
Frames

Frames Per Second : 4.201680672268908
Frames Per Second : 4.291845493562231
Frames Per Second : 4.405286343612334
Frames Per Second : 4.291845493562231
Frames Per Second : 4.329004329004329
Frames Per Second : 4.672897196261682
Frames Per Second : 4.329004329004329
Frames Per Second : 4.273504273504273
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.032258064516129
Frames Per Second : 4.464285714285714
Frames Per Second : 4.25531914893617
Frames Per Second : 3.7878787878787876
Frames Per Second : 3.731343283582089
Frames Per Second : 3.9840637450199203
Frames Per Second : 3.5211267605633805
Frames Per Second : 4.11522633744856
Frames Per Second : 4.329004329004329
Frames Per Second : 4.081632653061225
Frames Per Second : 3.9215686274509802
Frames Per Second : 3.773584905660377
Frames Per Second : 4.237288135593221
Frames Per Second : 3.9840637450199203
Frames Per Second : 4.219409282700422
Frames Per Second : 4.310344827586206
Frames Per Second : 4.149377593360996
Frames P

Frames Per Second : 4.237288135593221
Frames Per Second : 4.545454545454546
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.166666666666667
Frames Per Second : 4.444444444444445
Frames Per Second : 4.273504273504273
Frames Per Second : 4.566210045662101
Frames Per Second : 4.444444444444445
Frames Per Second : 4.065040650406504
Frames Per Second : 4.587155963302752
Frames Per Second : 4.25531914893617
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.424778761061947
Frames Per Second : 4.484304932735426
Frames Per Second : 4.672897196261682
Frames Per Second : 4.405286343612334
Frames Per Second : 4.405286343612334
Frames Per Second : 4.62962962962963
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.784688995215311
Frames Per Second : 4.524886877828054
Frames Per Second : 4.329004329004329
Frames Per Second : 4.524886877828054
Frames Per Second : 4.504504504504505
Frames Per Second : 4.484304932735426
Frames Per Second : 4.385964912280701
Frames Per 

Frames Per Second : 4.065040650406504
Frames Per Second : 4.201680672268908
Frames Per Second : 4.11522633744856
Frames Per Second : 4.25531914893617
Frames Per Second : 4.366812227074235
Frames Per Second : 4.366812227074235
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.0
Frames Per Second : 3.8167938931297707
Frames Per Second : 4.065040650406504
Frames Per Second : 4.184100418410042
Frames Per Second : 4.065040650406504
Frames Per Second : 4.329004329004329
Frames Per Second : 4.184100418410042
Frames Per Second : 4.098360655737705
Frames Per Second : 4.310344827586206
Frames Per Second : 4.504504504504505
Frames Per Second : 4.25531914893617
Frames Per Second : 4.048582995951417
Frames Per Second : 4.273504273504273
Frames Per Second : 4.201680672268908
Frames Per Second : 4.464285714285714
Frames Per Second : 4.219409282700422
Frames Per Second : 3.623188405797101
Frames Per Second : 3.875968992248062
Frames Per Second : 3.9215686274509802
Frames Per Second : 3.6231

Frames Per Second : 4.366812227074235
Frames Per Second : 4.545454545454546
Frames Per Second : 4.651162790697675
Frames Per Second : 4.444444444444445
Frames Per Second : 4.329004329004329
Frames Per Second : 4.672897196261682
Frames Per Second : 4.587155963302752
Frames Per Second : 4.587155963302752
Frames Per Second : 4.405286343612334
Frames Per Second : 4.098360655737705
Frames Per Second : 4.184100418410042
Frames Per Second : 4.310344827586206
Frames Per Second : 4.201680672268908
Frames Per Second : 4.424778761061947
Frames Per Second : 4.11522633744856
Frames Per Second : 4.484304932735426
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.11522633744856
Frames Per Second : 4.237288135593221
Frames Per Second : 4.219409282700422
Frames Per Second : 4.237288135593221
Frames Per Second : 4.184100418410042
Frames Per Second : 4.464285714285714
Frames Per Second : 4.545454545454546
Frames Per Second : 4.366812227074235
Frames Per Second : 4.504504504504505
Frames Per Se

Frames Per Second : 4.273504273504273
Frames Per Second : 4.329004329004329
Frames Per Second : 4.166666666666667
Frames Per Second : 4.504504504504505
Frames Per Second : 4.464285714285714
Frames Per Second : 4.587155963302752
Frames Per Second : 4.545454545454546
Frames Per Second : 4.385964912280701
Frames Per Second : 4.424778761061947
Frames Per Second : 4.484304932735426
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.484304932735426
Frames Per Second : 4.484304932735426
Frames Per Second : 4.464285714285714
Frames Per Second : 4.524886877828054
Frames Per Second : 4.366812227074235
Frames Per Second : 4.3478260869565215
Frames Per Second : 4.25531914893617
Frames Per Second : 4.484304932735426
Frames Per Second : 4.424778761061947
Frames Per Second : 4.6082949308755765
Frames Per Second : 4.405286343612334
Frames Per Second : 4.405286343612334
Frames Per Second : 4.385964912280701
Frames Per Second : 4.385964912280701
Frames Pe

AssertionError: 